## Using MCP with langchain

In [1]:
!pip install -q langgraph langchain-mcp-adapters langchain_google_genai pip install python-dotenv

In [2]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv

load_dotenv()

In [14]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
import asyncio

import os
# os.environ["GOOGLE_API_KEY"] = "ENTER_API_KEY_HERE"

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    # other params...
)

server_params = StdioServerParameters(
    command="python",
    args=["finance_mcp_server.py"]
)

async def run_agent(message):
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools=await load_mcp_tools(session)
            agent = create_react_agent(llm, tools)
            agent_response = await agent.ainvoke(
                {"messages": message}
            )
            return agent_response
    

In [15]:
from IPython.display import Markdown, display
def display_messages(result):
    """Display formatted messages"""
    for message in result["messages"]:
        display(Markdown(message.content))


In [16]:
question = "What's the revenue in 2024 from financial income statement for ticker TSLA"
result = asyncio.run(run_agent(question))
display_messages(result)

In [13]:
 display(Markdown(result["messages"][3].content))

## Using multiple MCP Server in langchain

In [18]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

async with MultiServerMCPClient(
    {
        "finance": {
            "command": "python",
            "args": ["finance_mcp_server.py"],
            "transport": "stdio",
        },
        "filesystem": {
            "command": "python",
            "args": ["filesystem_mcp_server.py"],
            "transport": "stdio",
        }
    }
) as client:
    agent = create_react_agent(llm, client.get_tools())
    question = "Get financial income statement for ticker TSLA and write to a file called tsla.json, to verfify read the file and display"
    response = await agent.ainvoke({"messages": question})


In [19]:
display_messages(response)